In [1]:
import calendar
import os
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
#from pandas.tseries.offsets import BDay
from pandas.tseries.offsets import * 

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.options.display.max_rows = 11
pd.options.display.float_format = '{:.2f}'.format

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"

today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2023, 7, 14), datetime.date(2023, 7, 13))

In [2]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
print(f'today: {today}')
to_mmdd = today.strftime('%m-%d')
print(f'Today in mm-dd format: {to_mmdd}')
print(f'yesterday: {yesterday}')

today: 2023-07-14
Today in mm-dd format: 07-14
yesterday: 2023-07-13 00:00:00


In [3]:
one_month_ago = yesterday + DateOffset(days = -28)
print('One month ago: ', one_month_ago, ' ', one_month_ago.day_name())

One month ago:  2023-06-15 00:00:00   Thursday


In [4]:
format_dict = {
    'ttl_qty':'{:,}','qty':'{:,}',
    'from_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}','fm_price':'{:.2f}',
    'price':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}',
    'maxp':'{:.2f}','minp':'{:.2f}','pct':'{:,.2f}%',  
}

cols = 'name fm_md to_md days fm_price to_price pct spd avg max_price min_price ttl_qty trend'.split()

### Get past one quarter data

In [5]:
sql = """
SELECT name
FROM buy 
ORDER BY name
"""
buy = pd.read_sql(sql, const)

names = buy['name'].values.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'AH', 'ASK', 'ASP', 'AWC', 'BBL', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'ICHI', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'MCS', 'NER', 'ORI', 'PTG', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TMT', 'WHAIR', 'WHART'"

In [6]:
in_p = in_p + ", 'BEM', 'EA'"
in_p

"'AH', 'ASK', 'ASP', 'AWC', 'BBL', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'ICHI', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'MCS', 'NER', 'ORI', 'PTG', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TMT', 'WHAIR', 'WHART', 'BEM', 'EA'"

### Bypass this statement if aboce statement is executed

In [7]:
sql = """
SELECT name
FROM orders
ORDER BY trade, name
"""
#buy = pd.read_sql(sql, conlite)

#names = buy['name'].values.tolist()
#in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'AH', 'ASK', 'ASP', 'AWC', 'BBL', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'ICHI', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'MCS', 'NER', 'ORI', 'PTG', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TMT', 'WHAIR', 'WHART', 'BEM', 'EA'"

### End of Selection between Buy or Orders

In [8]:
sql = """
SELECT name, date, price, qty, maxp, minp
FROM price 
WHERE date >= '%s' AND name IN (%s) 
ORDER BY name, date"""
sql = sql % (one_month_ago, in_p)
print(sql)


SELECT name, date, price, qty, maxp, minp
FROM price 
WHERE date >= '2023-06-15 00:00:00' AND name IN ('AH', 'ASK', 'ASP', 'AWC', 'BBL', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'ICHI', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'MCS', 'NER', 'ORI', 'PTG', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TMT', 'WHAIR', 'WHART', 'BEM', 'EA') 
ORDER BY name, date


In [9]:
price = pd.read_sql(sql, const)
price.sample(5)

,name,date,price,qty,maxp,minp
61,ASP,2023-07-10,2.78,189483,2.80,2.78
146,BEM,2023-07-05,8.50,13339801,8.60,8.50
320,JMART,2023-07-03,17.10,43547035,17.40,15.20
476,PTTGC,2023-07-05,38.75,20611379,39.25,38.25
467,PTTGC,2023-06-22,37.25,11320964,38.00,37.00


In [10]:
data_path = "../data/"
file_name = "Quarterly-Price-by-Name.csv"
output_file = data_path + file_name
osd_file = osd_path + file_name

price.to_csv(output_file, header=None, index=False)
price.to_csv(osd_file, header=True, index=False)

### Call ruby ruby\crt-summary.rb

In [11]:
%pwd

'C:\\Users\\User\\OneDrive\\A7\\Daily'

In [12]:
os.chdir("C:\\users\\user\\onedrive\\a7")
%pwd

'C:\\users\\user\\onedrive\\a7'

In [13]:
!ruby ruby\\crt-summary.rb

ruby/../Data/Quarterly-Price-by-Name.csv
name|fm_date|to_date|days|fm_price|to_price|diff|pct|spd|avg|max_price|min_price|trend|nbr|ttl_qty
AH|2023-06-15|2023-06-15|1|34.75|34.75|0.0|0.0|0|0|35.25|34.5||2|1,501,200
AH|2023-06-16|2023-06-21|4|34.5|32.75|-1.75|-5.07|-7|-2|35.0|32.25|Downward|6|8,367,251
AH|2023-06-22|2023-06-22|1|33.25|33.25|0.0|0.0|0|0|33.75|32.25|Upward|7|2,011,971
AH|2023-06-23|2023-06-26|2|32.25|30.5|-1.75|-5.43|-7|-4|33.0|30.5|Downward|9|6,611,429
AH|2023-06-27|2023-07-03|5|32.25|35.0|2.75|8.53|11|2|35.0|30.75|Upward|14|10,657,316
AH|2023-07-04|2023-07-04|1|34.5|34.5|0.0|0.0|0|0|35.0|34.25|Downward|15|689,571
AH|2023-07-05|2023-07-07|3|35.0|35.75|0.75|2.14|3|1|35.75|34.25|Upward|18|4,153,754
AH|2023-07-10|2023-07-12|3|35.0|33.0|-2.0|-5.71|-8|-3|35.5|33.0|Downward|21|4,158,197
AH|2023-07-13|2023-07-14|2|33.5|34.0|0.5|1.49|2|1|34.5|33.0|Upward|23|1,723,700
ASK|2023-06-15|2023-06-19|3|26.25|26.5|0.25|0.95|1|0|27.5|26.25|Upward|26|3,139,551
ASK|2023-06-20|2023-06-26|5|2

SCC|2023-06-28|2023-06-28|1|322.0|322.0|0.0|0.0|0|0|326.0|321.0|Upward|517|1,638,366
SCC|2023-06-29|2023-06-29|1|320.0|320.0|0.0|0.0|0|0|324.0|320.0|Downward|518|1,431,061
SCC|2023-06-30|2023-07-05|4|322.0|327.0|5.0|1.55|5|1|329.0|320.0|Upward|522|7,881,662
SCC|2023-07-06|2023-07-07|2|323.0|322.0|-1.0|-0.31|-1|-1|326.0|320.0|Downward|524|3,471,976
SCC|2023-07-10|2023-07-10|1|324.0|324.0|0.0|0.0|0|0|325.0|321.0|Upward|525|1,197,179
SCC|2023-07-11|2023-07-11|1|323.0|323.0|0.0|0.0|0|0|325.0|322.0|Downward|526|1,207,052
SCC|2023-07-12|2023-07-12|1|326.0|326.0|0.0|0.0|0|0|326.0|323.0|Upward|527|1,945,871
SCC|2023-07-13|2023-07-13|1|323.0|323.0|0.0|0.0|0|0|327.0|323.0|Downward|528|1,412,382
SCC|2023-07-14|2023-07-14|1|328.0|328.0|0.0|0.0|0|0|328.0|323.0|Upward|529|1,818,083
SENA|2023-06-15|2023-06-20|4|3.12|3.12|0.0|0.0|0|0|3.14|3.08|Upward|533|2,512,352
SENA|2023-06-21|2023-06-28|6|3.08|2.9|-0.18|-5.84|-9|-2|3.12|2.9|Downward|539|4,218,838
SENA|2023-06-29|2023-07-03|3|2.96|3.02|0.06|2.03|3|

In [14]:
os.chdir("C:\\users\\user\\onedrive\\a7\\daily")
%pwd

'C:\\users\\user\\onedrive\\a7\\daily'

In [15]:
file_name   = 'summary.csv'
input_file = data_path + file_name
df = pd.read_csv(input_file, sep=',', index_col=None)
df.sample(5).style.format(format_dict)

,name,from_date,to_date,days,from_price,to_price,diff,pct,spd,avg,max_price,min_price,ttl_qty,trend,nbr
27,AWC,2023-06-28,2023-06-30,3,4.78,4.94,0.16,3.35%,8,2,4.98,4.76,"122,046,556",Upward,79
153,MCS,2023-06-30,2023-07-05,4,6.35,6.70,0.35,5.51%,7,1,6.75,6.20,"1,684,231",Upward,390
76,DIF,2023-06-27,2023-07-03,5,10.70,11.10,0.40,3.74%,4,0,11.20,10.60,"95,005,773",Upward,190
226,SINGER,2023-07-05,2023-07-12,6,9.25,8.75,-0.50,-5.41%,-10,-2,9.80,8.70,"35,804,932",Downward,571
68,CPNREIT,2023-06-28,2023-07-04,5,10.80,10.70,-0.10,-0.93%,-1,-1,11.40,10.50,"18,617,404",Downward,169


In [16]:
df.rename(columns={'from_date':'fm_date','from_price':'fm_price'},inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df['fm_md'] = df['fm_date'].dt.strftime('%m-%d')
df['to_md'] = df['to_date'].dt.strftime('%m-%d')

### Group analysis

In [17]:
mask_p = df['to_md'] == to_mmdd
df[mask_p][cols].sort_values('pct',ascending=False).head().style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
150,KCE,07-13,07-14,2,36.75,38.75,5.44%,8,4,39.25,35.75,"21,452,872",Upward
89,EA,07-12,07-14,3,53.75,56.25,4.65%,10,3,56.50,52.75,"34,593,396",Upward
185,PTTGC,07-10,07-14,5,39.00,40.50,3.85%,6,1,40.75,38.50,"56,860,054",Upward
105,ICHI,07-13,07-14,2,13.30,13.80,3.76%,5,2,13.80,12.90,"14,246,856",Upward
110,IVL,07-13,07-14,2,34.00,35.25,3.68%,5,2,35.25,33.00,"38,351,135",Upward


In [18]:
name_up = 'CPNREIT'
mask_n = df.name == name_up
df[mask_n][cols].style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
63,CPNREIT,06-15,06-15,1,12.10,12.10,0.00%,0,0,12.60,11.80,"6,556,341",Upward
64,CPNREIT,06-16,06-21,4,12.00,10.10,-15.83%,-19,-5,12.30,10.10,"32,720,704",Downward
65,CPNREIT,06-22,06-23,2,10.40,11.00,5.77%,6,3,11.00,10.20,"10,290,298",Upward
66,CPNREIT,06-26,06-26,1,10.70,10.70,0.00%,0,0,11.00,10.70,"5,994,644",Downward
67,CPNREIT,06-27,06-27,1,11.40,11.40,0.00%,0,0,11.90,11.20,"12,987,769",Upward
68,CPNREIT,06-28,07-04,5,10.80,10.70,-0.93%,-1,-1,11.40,10.50,"18,617,404",Downward
69,CPNREIT,07-05,07-10,4,10.80,11.00,1.85%,2,0,11.00,10.70,"11,304,068",Upward
70,CPNREIT,07-11,07-11,1,10.80,10.80,0.00%,0,0,11.00,10.70,"3,715,262",Downward
71,CPNREIT,07-12,07-12,1,10.90,10.90,0.00%,0,0,10.90,10.80,"1,482,408",Upward
72,CPNREIT,07-13,07-13,1,10.80,10.80,0.00%,0,0,10.90,10.80,"583,201",Downward


In [19]:
mask_m = df['to_md'] == to_mmdd
df[mask_m][cols].sort_values('pct',ascending=True).head().style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
77,DIF,07-04,07-14,9,11.00,10.70,-2.73%,-3,-1,11.20,10.50,"84,070,184",Downward
287,WHART,07-14,07-14,1,10.70,10.70,0.00%,0,0,10.70,10.60,"1,013,328",Upward
211,SCC,07-14,07-14,1,328.00,328.00,0.00%,0,0,328.00,323.00,"1,818,083",Upward
274,WHAIR,07-13,07-14,2,7.05,7.05,0.00%,0,0,7.05,7.00,"269,160",Upward
216,SENA,07-14,07-14,1,3.00,3.00,0.00%,0,0,3.00,2.94,"451,000",Upward


In [20]:
name_down = 'PTG'
mask_n = df.name == name_down
df[mask_n][cols].style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
174,PTG,06-15,06-16,2,12.10,12.20,0.83%,1,0,12.30,12.10,"5,677,772",Upward
175,PTG,06-19,06-28,8,12.00,11.60,-3.33%,-4,-1,12.30,11.30,"39,404,185",Downward
176,PTG,06-29,07-03,3,12.20,12.30,0.82%,1,0,12.30,11.50,"18,944,789",Upward
177,PTG,07-04,07-13,8,12.00,10.10,-15.83%,-19,-3,12.30,10.40,"99,138,422",Downward
178,PTG,07-14,07-14,1,10.20,10.20,0.00%,0,0,10.30,10.00,"12,710,643",Upward


In [21]:
df_groupby_trend = df.groupby('trend')

In [22]:
# To retrieve one of the created groups
df_up = df_groupby_trend.get_group('Upward')
df_up[cols].sample(5).style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
99,ICHI,06-15,06-15,1,15.00,15.00,0.00%,0,0,15.40,15.00,"4,615,457",Upward
254,TFFIF,06-27,06-28,2,7.60,7.65,0.66%,1,0,7.65,7.50,"5,146,038",Upward
16,ASP,06-15,06-20,4,2.84,2.86,0.70%,1,0,2.88,2.82,"3,046,081",Upward
165,ORI,06-20,06-20,1,11.10,11.10,0.00%,0,0,11.20,10.90,"2,159,132",Upward
181,PTTGC,06-19,06-20,2,38.50,38.75,0.65%,1,0,39.00,37.50,"24,449,802",Upward


In [23]:
# To retrieve one of the created groups
df_dw = df_groupby_trend.get_group('Downward')
df_dw[cols].sample(5).style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
215,SENA,07-04,07-13,8,3.00,2.98,-0.67%,-1,-1,3.02,2.92,"2,309,314",Downward
246,SYNEX,07-12,07-13,2,11.70,11.70,0.00%,0,0,12.10,11.40,"701,986",Downward
26,AWC,06-21,06-27,5,4.90,4.76,-2.86%,-7,-2,5.05,4.74,"142,488,515",Downward
30,AWC,07-05,07-07,3,4.82,4.60,-4.56%,-11,-4,4.94,4.54,"133,658,043",Downward
112,JASIF,06-19,06-26,6,7.25,7.00,-3.45%,-5,-1,7.45,7.00,"50,327,385",Downward


In [24]:
# there is a function called .agg() and it allows us to specify multiple aggregation functions at once
df.groupby('trend').days.agg(['max', 'count', 'median', 'mean'])

,max,count,median,mean
trend,,,,
Downward,9,130,2.00,3.01
Upward,7,157,2.00,1.99


In [25]:
upward_max = df.groupby('trend').days.max().loc['Upward']
downward_max = df.groupby('trend').days.max().loc['Downward']

In [26]:
df.query('days == @upward_max and trend == "Upward"')[cols].style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
256,TFFIF,07-03,07-11,7,7.60,7.60,0.00%,0,0,7.65,7.55,"19,152,453",Upward


In [27]:
df.query('days == @downward_max and trend == "Downward"')[cols]

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
77,DIF,07-04,07-14,9,11.00,10.70,-2.73,-3,-1,11.20,10.50,84070184,Downward


### Individual stock analysis

In [28]:
df.query('name == @name_up')[cols].style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
63,CPNREIT,06-15,06-15,1,12.10,12.10,0.00%,0,0,12.60,11.80,"6,556,341",Upward
64,CPNREIT,06-16,06-21,4,12.00,10.10,-15.83%,-19,-5,12.30,10.10,"32,720,704",Downward
65,CPNREIT,06-22,06-23,2,10.40,11.00,5.77%,6,3,11.00,10.20,"10,290,298",Upward
66,CPNREIT,06-26,06-26,1,10.70,10.70,0.00%,0,0,11.00,10.70,"5,994,644",Downward
67,CPNREIT,06-27,06-27,1,11.40,11.40,0.00%,0,0,11.90,11.20,"12,987,769",Upward
68,CPNREIT,06-28,07-04,5,10.80,10.70,-0.93%,-1,-1,11.40,10.50,"18,617,404",Downward
69,CPNREIT,07-05,07-10,4,10.80,11.00,1.85%,2,0,11.00,10.70,"11,304,068",Upward
70,CPNREIT,07-11,07-11,1,10.80,10.80,0.00%,0,0,11.00,10.70,"3,715,262",Downward
71,CPNREIT,07-12,07-12,1,10.90,10.90,0.00%,0,0,10.90,10.80,"1,482,408",Upward
72,CPNREIT,07-13,07-13,1,10.80,10.80,0.00%,0,0,10.90,10.80,"583,201",Downward


In [29]:
df_tmp = price.query('name == @name_up')
df_tmp.describe().round(2)

,price,qty,maxp,minp
count,22.00,22.00,22.00,22.00
mean,10.92,4793575.86,11.16,10.73
std,0.45,3837656.56,0.54,0.46
min,10.10,583201.00,10.50,9.90
25%,10.80,1988936.00,10.90,10.62
50%,10.80,3618795.00,11.00,10.70
75%,11.00,5972644.25,11.08,10.80
max,12.10,13675377.00,12.60,11.90


In [30]:
df.query('name == @name_down')[cols].style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
174,PTG,06-15,06-16,2,12.10,12.20,0.83%,1,0,12.30,12.10,"5,677,772",Upward
175,PTG,06-19,06-28,8,12.00,11.60,-3.33%,-4,-1,12.30,11.30,"39,404,185",Downward
176,PTG,06-29,07-03,3,12.20,12.30,0.82%,1,0,12.30,11.50,"18,944,789",Upward
177,PTG,07-04,07-13,8,12.00,10.10,-15.83%,-19,-3,12.30,10.40,"99,138,422",Downward
178,PTG,07-14,07-14,1,10.20,10.20,0.00%,0,0,10.30,10.00,"12,710,643",Upward


In [31]:
df_tmp = price.query('name == @name_down')
df_tmp.describe().round(2)

,price,qty,maxp,minp
count,22.00,22.00,22.00,22.00
mean,11.65,7994355.05,11.87,11.50
std,0.62,10539190.20,0.55,0.63
min,10.10,2120526.00,10.30,9.90
25%,11.60,2788030.25,11.72,11.42
50%,11.70,4733039.50,11.95,11.60
75%,12.00,6831095.25,12.28,11.90
max,12.30,47072508.00,12.30,12.10


In [32]:
mask_p = df['to_md'] == to_mmdd
summary = df[mask_p][cols].sort_values(['pct','trend'],ascending=[False,False])

In [33]:
file_name = 'summary-240.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

summary[cols].to_csv(output_file, header=True, index=False)
summary[cols].to_csv(data_file, header=True, index=False)
summary[cols].to_csv(one_file, header=True, index=False)
summary[cols].to_csv(osd_file, header=True, index=False)

In [34]:
sql = """
SELECT name, date, price, qty, maxp, minp
FROM price 
WHERE date >= '%s' 
ORDER BY name, date"""
sql = sql % (today)
print(sql)


SELECT name, date, price, qty, maxp, minp
FROM price 
WHERE date >= '2023-07-14' 
ORDER BY name, date


In [35]:
df = pd.read_sql(sql, const)
df

,name,date,price,qty,maxp,minp
0,ACE,2023-07-14,1.84,18372807,1.85,1.80
1,ADVANC,2023-07-14,222.00,6238281,222.00,218.00
2,AEONTS,2023-07-14,164.00,862765,168.00,162.00
3,AH,2023-07-14,34.00,1150979,34.50,33.25
4,AIMIRT,2023-07-14,11.50,68400,11.50,11.40
...,...,...,...,...,...,...
182,WHAIR,2023-07-14,7.05,153401,7.05,7.00
183,WHART,2023-07-14,10.70,1013328,10.70,10.60
184,WHAUP,2023-07-14,3.56,640805,3.58,3.54
185,WICE,2023-07-14,8.55,516007,8.55,8.35
